# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv C:\Users\arezo\DSI_Assignments\production\production\01_materials\labs\.env

In [2]:
import dask.dataframe as dd

c:\Users\arezo\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\arezo\AppData\Local\Temp\ipykernel_12624\676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Set the directory path
directory_path = '../../05_src/data/prices'
PRICE_DATA = os.getenv('PRICE_DATA', directory_path)

# Use glob to find all parquet files recursively
parquet_files = glob(os.path.join(PRICE_DATA, '*/*/*.0.parquet'))

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

c:\Users\arezo\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


In [4]:
len(parquet_files)

11207

In [5]:
# Display the first few rows of the Dask DataFrame

dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463028,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142944,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652863,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219181,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237450,2819626,Health Care,Life Sciences Tools & Services,2000


In [6]:
# Display columns' names

print('columns_in_dataframe: ', dd_px.columns)

columns_in_dataframe:  Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [8]:
# Write your code below.

dd_feat = (dd_px.groupby('ticker', group_keys=False)
           .apply(lambda x: x.assign(
               Adj_Close_lag=x['Adj Close'].shift(1),
               Close_lag=x['Close'].shift(1)))
           .assign(
               returns=lambda x: x['Adj Close'] / x['Adj_Close_lag'] - 1,
               hi_lo_range=lambda x: x['High'] - x['Low'])
          )

C:\Users\arezo\AppData\Local\Temp\ipykernel_12624\201419781.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [9]:
# Add a rolling average return calculation with a window of 10 days.

return_df = (dd_feat
             .compute()
             .groupby('ticker', group_keys = False)
             .apply(lambda x: x.assign(ma_return = x['returns'].rolling(10).mean())))
return_df

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Adj_Close_lag,Close_lag,returns,hi_lo_range,ma_return
ticker,,,,,,,,,,,,,,,
HUM,2015-01-02,144.949997,146.509995,141.389999,142.990005,133.404572,982800,Health Care,Managed Health Care,2015,NaN,NaN,NaN,5.119995,NaN
HUM,2015-01-05,141.690002,142.759995,138.690002,139.199997,129.868637,1686700,Health Care,Managed Health Care,2015,133.404572,142.990005,-0.026505,4.069992,NaN
HUM,2015-01-06,140.210007,141.490005,137.449997,139.089996,129.765945,1770100,Health Care,Managed Health Care,2015,129.868637,139.199997,-0.000791,4.040009,NaN
HUM,2015-01-07,139.729996,142.240005,139.729996,141.729996,132.228989,877400,Health Care,Managed Health Care,2015,129.765945,139.089996,0.018981,2.510010,NaN
HUM,2015-01-08,143.619995,148.389999,143.619995,147.830002,137.920120,1350000,Health Care,Managed Health Care,2015,132.228989,141.729996,0.043040,4.770004,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2013-12-24,21.469999,21.889999,21.469999,21.809999,16.996265,2733000,Information Technology,Communications Equipment,2013,16.894957,21.680000,0.005996,0.420000,0.001142
JNPR,2013-12-26,21.860001,22.000000,21.809999,21.910000,17.074194,2942800,Information Technology,Communications Equipment,2013,16.996265,21.809999,0.004585,0.190001,0.002851
JNPR,2013-12-27,21.940001,22.299999,21.910000,22.219999,17.315765,3647100,Information Technology,Communications Equipment,2013,17.074194,21.910000,0.014148,0.389999,0.007455


In [10]:
# Add a rolling average return calculation with a window of 10 days without converting to panda.

return_df_da = (dd_feat
             .groupby('ticker', group_keys = False)
             .apply(lambda x: x.assign(ma_return = x['returns'].rolling(10).mean())))
return_df_da

C:\Users\arezo\AppData\Local\Temp\ipykernel_12624\3670255570.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  return_df_da = (dd_feat


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Adj_Close_lag,Close_lag,returns,hi_lo_range,ma_return
npartitions=11207,,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
return_df_da.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Adj_Close_lag,Close_lag,returns,hi_lo_range,ma_return
ticker,,,,,,,,,,,,,,,
HUM,2015-01-02,144.949997,146.509995,141.389999,142.990005,133.404572,982800,Health Care,Managed Health Care,2015,NaN,NaN,NaN,5.119995,NaN
HUM,2015-01-05,141.690002,142.759995,138.690002,139.199997,129.868637,1686700,Health Care,Managed Health Care,2015,133.404572,142.990005,-0.026505,4.069992,NaN
HUM,2015-01-06,140.210007,141.490005,137.449997,139.089996,129.765945,1770100,Health Care,Managed Health Care,2015,129.868637,139.199997,-0.000791,4.040009,NaN
HUM,2015-01-07,139.729996,142.240005,139.729996,141.729996,132.228989,877400,Health Care,Managed Health Care,2015,129.765945,139.089996,0.018981,2.510010,NaN
HUM,2015-01-08,143.619995,148.389999,143.619995,147.830002,137.920120,1350000,Health Care,Managed Health Care,2015,132.228989,141.729996,0.043040,4.770004,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2013-12-24,21.469999,21.889999,21.469999,21.809999,16.996265,2733000,Information Technology,Communications Equipment,2013,16.894957,21.680000,0.005996,0.420000,0.001142
JNPR,2013-12-26,21.860001,22.000000,21.809999,21.910000,17.074194,2942800,Information Technology,Communications Equipment,2013,16.996265,21.809999,0.004585,0.190001,0.002851
JNPR,2013-12-27,21.940001,22.299999,21.910000,22.219999,17.315765,3647100,Information Technology,Communications Equipment,2013,17.074194,21.910000,0.014148,0.389999,0.007455


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

For smaller datasets, Pandas provides speed and simplicity.
For larger-than-memory datasets, Dask offers scalability and better memory management, making it more suitable for complex workflows where data size could exceed memory.

1) Was it necessary to convert to pandas to calculate the moving average return?
In my specific case, converting to Pandas before calculating the moving average return was more efficient, as demonstrated by the faster runtime. It wasn’t strictly necessary to use Pandas, but doing so significantly reduced processing time. The primary reason is that Pandas is optimized for in-memory operations, making it faster for calculations on small to medium datasets that fit in memory. 

2) Would it have been better to do it in Dask? Why?
In this case, no, Dask was not better for calculating the moving average because the calculation was slower due to Dask's partition management and task scheduling overhead. Dask is designed to handle large, out-of-memory data by breaking computations into chunks, but this can slow down certain operations—especially when performing complex, multi-step calculations like groupby combined with rolling. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.